This notebook contains some code to show how to run the Python files containing the model code.

In [1]:
import math, os

from Code.Data import *
from Code.Load import *
from Code.CP import *
from Code.QualityMeasures import *

import numpy as np
from tqdm.notebook import tqdm

In [2]:
data = "residential"

In [3]:
def summary(data, model, CP = True, folder = FOLDER):
    folder = FOLDER + "/" + data + "/results/"
    files = os.listdir(folder)
    cp = "-CP-" if CP else ""
    filtered = [file for file in files if (model+cp in file)]
    
    data = np.zeros((len(filtered), 3))
    
    for i, f in enumerate(filtered):
        with open(folder + f, "r") as file:
            parts = file.read().splitlines()[0].split("\t")
            data[i, 0] = float(parts[0])
            data[i, 1] = float(parts[1])
            data[i, 2] = float(parts[2])
            
    return {"cov": np.mean(data[:, 0]), "width": np.mean(data[:, 1]), "r2": np.mean(data[:, 2]), \
           "cov_std": np.std(data[:, 0]), "width_std": np.std(data[:, 1]), "r2_std": np.std(data[:, 2])}

In [4]:
import Code.RunNNCP as NN

for s in tqdm(SEEDS):
    NN.run(data, s)
    
summary(data, "NN")

  0%|          | 0/50 [00:00<?, ?it/s]

{'cov': 0.9079999999999999,
 'width': 0.9056779632,
 'r2': 0.9048712993550034,
 'cov_std': 0.04643753271271466,
 'width_std': 0.21845542104080634,
 'r2_std': 0.049514419759247356}

In [5]:
import Code.RunRFCP as RF

for s in tqdm(SEEDS):
    RF.run(data, s)
    
summary(data, "RF")

  0%|          | 0/50 [00:00<?, ?it/s]

{'cov': 0.8933333333333331,
 'width': 0.70241830013543,
 'r2': 0.922537051582105,
 'cov_std': 0.04866210024238575,
 'width_std': 0.13960731639180107,
 'r2_std': 0.03272888216455605}

In [6]:
import Code.RunDropout as Drop

for s in tqdm(SEEDS):
    Drop.run(data, s, num = 50, drop = np.arange(0.05, 0.55, 0.05), l = 0)
    
summary(data, "Dropout")

  0%|          | 0/50 [00:00<?, ?it/s]

{'cov': 0.9008000000000002,
 'width': 0.9242585380964167,
 'r2': 0.8961015631331557,
 'cov_std': 0.04301193632779936,
 'width_std': 0.16939045264018196,
 'r2_std': 0.051641656606872754}

In [7]:
import Code.RunKG as KG

for s in tqdm(SEEDS):
    KG.run(data, s, num = 50, drop = np.arange(0.05, 0.55, 0.05), l = 0)
    
summary(data, "KG")

  0%|          | 0/50 [00:00<?, ?it/s]

{'cov': 0.9048,
 'width': 0.7027188658593033,
 'r2': 0.932814930490126,
 'cov_std': 0.041825882470600005,
 'width_std': 0.1326416223991968,
 'r2_std': 0.03199325342906429}

In [8]:
import Code.RunDE as DE

for s in tqdm(SEEDS):
    DE.run(data, s, num = 5, drop = 0)
    
summary(data, "DE")

  0%|          | 0/50 [00:00<?, ?it/s]

{'cov': 0.9104000000000001,
 'width': 0.8224747525792108,
 'r2': 0.9028515348578716,
 'cov_std': 0.032989156467609845,
 'width_std': 0.3044602545752964,
 'r2_std': 0.07652361835704996}

In [9]:
import Code.RunQR as QR

for s in tqdm(SEEDS):
    QR.run(data, s)
    
summary(data, "QR")

  0%|          | 0/50 [00:00<?, ?it/s]

{'cov': 0.9079999999999999,
 'width': 0.858678389,
 'r2': 0.9178291127827555,
 'cov_std': 0.04424176006143215,
 'width_std': 0.1681320877292204,
 'r2_std': 0.05255667591025171}

In [10]:
import Code.RunGP as GP

for s in tqdm(SEEDS):
    GP.run(data, s, epochs = 50, learning_rate = 0.1)

summary(data, "GP")

  0%|          | 0/50 [00:00<?, ?it/s]

{'cov': 0.8959999999999998,
 'width': 2.32508633,
 'r2': 0.21690771303946296,
 'cov_std': 0.04474123129483337,
 'width_std': 0.30926339817376075,
 'r2_std': 0.07110859876654017}

In [ ]:
import Code.RunSVGP as AGP

for s in tqdm(SEEDS):
    AGP.run(data, s, learning_rate = 0.01, epochs = 50)
    
summary(data, "AGP")

  0%|          | 0/50 [00:00<?, ?it/s]